In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import rosbag
import rospy
import pandas as pd

import message_filters
from sensor_msgs.msg import Image
# from quadrotor_msgs.msg import PositionCommand
from nav_msgs.msg import Odometry

rospy.init_node('nn')

In [2]:
bag = rosbag.Bag('./first_train_data.bag')
topics = ['/pcl_render_node/depth', '/planning/pos_cmd', '/visual_slam/odom']
# column_names = ['pos_x', 'pos_y', 'pos_z', 'vel_x', 'vel_y', 'vel_z', 'acc_x', 'acc_y', 'acc_z', 'yaw', 'yaw_dot']
# df = pd.DataFrame(columns=column_names)

In [ ]:
def callback(depth, odom):
    print(odom)

depth_sub = message_filters.Subscriber('/pcl_render_node/depth', Image, callback)
# pos_cmd_sub = message_filters.Subscriber('/planning/pos_cmd', PositionCommand, callback)
odom_sub = message_filters.Subscriber('/visual_slam/odom', Odometry, callback)
ts.registerCallback(callback)
rospy.spin()

# for topic, msg, t in bag.read_messages(topics=topics):
#     if topic == topics[0]: # /pcl_render_node/depth
#         # height = 480
#         # width = 640

In [2]:
bag = rosbag.Bag('./2020-09-29-16-59-02.bag')
topic = '/planning/pos_cmd'
column_names = ['pos_x', 'pos_y', 'pos_z', 'vel_x', 'vel_y', 'vel_z', 'acc_x', 'acc_y', 'acc_z', 'yaw', 'yaw_dot']
df = pd.DataFrame(columns=column_names)

In [4]:
for topic, msg, t in bag.read_messages(topics=topic):
    df = df.append(
        {
            'pos_x': msg.position.x,
            'pos_y': msg.position.y,
            'pos_z': msg.position.z,
            'vel_x': msg.velocity.x,
            'vel_y': msg.velocity.y,
            'vel_z': msg.velocity.z,
            'acc_x': msg.acceleration.x,
            'acc_y': msg.acceleration.y,
            'acc_z': msg.acceleration.z,
            'yaw': msg.yaw,
            'yaw_dot': msg.yaw_dot
        },
        ignore_index=True
    )

df
# df.to_csv('pos_cmd.csv')

,pos_x,pos_y,pos_z,vel_x,vel_y,vel_z,acc_x,acc_y,acc_z,yaw,yaw_dot
0,-13.086069,0.554909,1.451747,1.974760,0.870654,0.406779,0.047508,0.687159,-0.476178,0.546454,0.641180
1,-13.101905,0.547797,1.448605,1.974918,0.876810,0.402332,0.016274,0.655919,-0.476710,0.530739,-1.250206
2,-13.082168,0.556591,1.452602,1.974917,0.883202,0.397565,-0.016428,0.623326,-0.477326,0.516868,-2.013050
3,-13.062407,0.565460,1.456557,1.974589,0.889276,0.392785,-0.049175,0.590689,-0.477944,0.511014,-1.591590
4,-13.042663,0.574382,1.460461,1.973934,0.895020,0.388002,-0.081900,0.558073,-0.478560,0.509179,-0.979292
...,...,...,...,...,...,...,...,...,...,...,...
10395,-14.902148,-0.069173,0.999636,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.577875,0.000000
10396,-14.902148,-0.069173,0.999636,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.577875,0.000000
10397,-14.902148,-0.069173,0.999636,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.577875,0.000000
10398,-14.902148,-0.069173,0.999636,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.577875,0.000000


In [ ]:
model = Sequential()
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=10))

train_X = [[1] * 100] * 100
test_X = [[1] * 100] * 100

train_Y = [[1] * 10] * 100
test_Y = [[1] * 10] * 100

# print(test_Y)

opt = SGD(lr = .1, momentum = 0.9)

model.compile(loss='mean_squared_error',
    optimizer=opt,
    metrics=['mean_squared_error'])

model.fit(train_X, train_Y, epochs=100, batch_size=32)

loss_and_metrics = model.evaluate(test_X, test_Y, batch_size=32)